In [3]:
import cv2
import numpy as np

In [4]:
cap = cv2.VideoCapture('./Hand Video2.mov')
bgs = cv2.createBackgroundSubtractorKNN(detectShadows=False)
    
if not cap.isOpened():
    print("Camera open failed!")
    exit()

while True:
    # 카메라로부터 프레임을 정상적으로 받아오면 ret에는 True,
    # frame에는 해당 프레임이 저장됨
    ret, frame = cap.read()
    
    if not ret:
        break
    
    # 현재 프레임 반전
    resize_frame = cv2.resize(frame, (0,0), fx=0.5, fy=0.5, interpolation = cv2.INTER_AREA)
    
    # skin color 이용해서 추출
    hsvmask = cv2.cvtColor(resize_frame, cv2.COLOR_BGR2HSV)
    skinmask = cv2.inRange(hsvmask, np.array([0, 48, 80], dtype = "uint8"), np.array([20, 255, 255], dtype = "uint8"))
    
    # motion difference 이용해서 추출
    fgmask = bgs.apply(resize_frame)
    mask = np.zeros((fgmask.shape[0]+2, fgmask.shape[1]+2), np.uint8)
    fgmask_fill = fgmask.copy()
    cv2.floodFill(fgmask_fill, mask, (0,0), 255)
    fgmask_fill_inv = cv2.bitwise_not(fgmask_fill)
    
    hsv_plus_motion = fgmask | fgmask_fill_inv & skinmask
    
    #dilation, erosion 전 마지막 salt-pepper noise 제거
    hsv_plus_motion = cv2.medianBlur(hsv_plus_motion, 5)
    
    #dilation으로 채운 후 erosion으로 줄이기
    kernel = np.ones((3,3), np.uint8)
    mask_final = cv2.erode(cv2.dilate(hsv_plus_motion, kernel, iterations=3), kernel, iterations=3)
    
    #채널별로 엣지까지 제거한 결과
    final_B = (mask_final-cv2.Canny(mask_final, 0, 360))&resize_frame[:,:,0]
    final_G = (mask_final-cv2.Canny(mask_final, 0, 360))&resize_frame[:,:,1]
    final_R = (mask_final-cv2.Canny(mask_final, 0, 360))&resize_frame[:,:,2]
    
    #medianBlur 추가해서 salt noise 제거
    final = cv2.merge((final_B,final_G,final_R))
    
    cv2.imshow('Hand Video', final)
    
    # 일정 시간 기다린 후 다음 프레임 처리.
    # 만약 ESC 키를 누르면 while 루프 종료
    if cv2.waitKey(10) == 27:
        break

# 사용한 자원 해제
cap.release()
cv2.destroyAllWindows()